In [147]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pygame
import time
import cv2
import h5py

LETTER_GESTURES = {
    "a": "Swipe Forward",
    "b": "Swipe Backward",
    "c": "Swipe Left",
    "d": "Swipe Right",
    "p": "Fast Pinch",
    "prr": "Rotate Right",
    "prl": "Rotate Left",
    "pbd": "Back to Default",
    "pc": "Pinch Hold",
    "po": "Pinch Open",
    "sp": "Side Pinch",
    "o": "Nothing",
    "s": "Knock"
}

In [38]:
def show_video_from_frames_pygame(frames, fps=30):
    """
    Displays video frames using PyGame, overlays frame IDs, and maintains specified FPS.
    
    Args:
        frames (list): List of video frames (BGR format).
        fps (int): Frames per second for playback.
    
    Returns:
        bool: True if the video plays without interruption; False if 'q' is pressed to quit.
    """
    # Initialize PyGame
    
    if isinstance(frames[0], tuple):
        frame_times = [frame[1] for frame in frames]
        frames = [frame[0] for frame in frames]
    
    pygame.init()
    clock = pygame.time.Clock()

    # Set up the display window
    frame_height, frame_width = frames[0].shape[:2]
    #print(frame_height, frame_width)
    screen = pygame.display.set_mode((frame_width, frame_height))
    pygame.display.set_caption("Video Playback")

    running = True
    for frame_id, frame in enumerate(frames):
        
        # Handle events (exit on quit or 'q' key)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            if event.type == pygame.KEYDOWN and event.key == pygame.K_q:
                running = False
        
        if not running:
            break

        # Convert the frame from BGR (OpenCV format) to RGB (PyGame format)
        #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Overlay the frame ID on the frame
        text = f"Frame: {frame_id}"
        font = cv2.FONT_HERSHEY_SIMPLEX
        position = (200, 30)
        font_scale = 1
        font_color = (255, 255, 255)  # White
        thickness = 2
        line_type = cv2.LINE_AA
        cv2.putText(frame, text, position, font, font_scale, font_color, thickness, line_type)

        # Convert frame to a surface for PyGame
        frame_surface = pygame.surfarray.make_surface(np.transpose(frame, (1,0,2)))

        # Display the frame on the PyGame window
        screen.blit(frame_surface, (0, 0))
        pygame.display.flip()

        # Wait to maintain the desired FPS
        clock.tick(fps)

    # Quit PyGame
    pygame.quit()
    return running


def load_frames_by_time_range(video_file, start_timestamp, end_timestamp, offset=0, delay_frames=0):
    cap = cv2.VideoCapture(video_file)
    frames = []
    
    timestamps_file = video_file.replace(".avi", "_timestamps.txt")
    start_time, end_time = None, None
    for line in open(timestamps_file, "r").readlines():
        if line.startswith("Recording Start Time:"):
            start_time = float(line.split(":")[-1])
        if line.startswith("Recording End Time:"):
            end_time = float(line.split(":")[-1])
    
    assert start_time is not None and end_time is not None, "Could not read start and end time from file"
    
    # Get video FPS and total number of frames
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate delay time and adjust start and end timestamps
    delay_time = delay_frames * 1 / fps  # Delay in seconds
    start_time += delay_time
    end_time += delay_time

    # Calculate the start and end frame indices
    start_frame = int((start_timestamp - start_time) * fps) + offset
    end_frame = int((end_timestamp - start_time) * fps) + offset

    # Ensure the frame indices are within bounds
    start_frame = max(0, start_frame)
    end_frame = min(total_frames, end_frame)

    # Seek to the start frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    # Read frames sequentially
    for frame_index in range(start_frame, end_frame + 1):
        ret, frame = cap.read()
        if not ret:
            break  # Stop if we can't read the frame
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_time = start_time + frame_index / fps
        frames.append((frame, frame_time))
    
    cap.release()
    return frames

In [21]:
bad_preds_path = r"C:\Users\lhauptmann\Code\WristPPG2\scripts\bad_preds_fold1.csv"
bad_preds_df = pd.read_csv(bad_preds_path, index_col=0)

participants_with_video = ['Daria', 'Paula', 'Hannah', 'Elea', 'Nicole', 'Julia', 'Sam', 'Miro', 'Amran', 'Caroline', 'Davud', 'Anusha', 'Frawa', 'Pam', 'Miyuki', 'Maedeh', 'Eren', 'Bo', 'Moritz', 'Blaise', 'Rolando', 'Oscar', 'Toby', 'Alex', 'Lilian', 'Gonzalo']
participants_with_video = [el.lower() for el in participants_with_video]

In [211]:
bad_preds_df["participant"] = bad_preds_df["sample_id"].apply(lambda x: x.split("/")[-2].split("_")[1].lower())
bad_preds_df["gesture_id"] = bad_preds_df["sample_id"].apply(lambda x: x.split("/")[-1].replace("']", ""))
bad_preds_df["session"] = bad_preds_df["sample_id"].apply(lambda x: x.split("/")[-2].split("_")[2].replace(".hdf5", ""))
bad_preds_df["gesture"] = bad_preds_df["gesture_id"].apply(lambda x: x.split("_")[0])
bad_preds_df["filename"] = bad_preds_df["sample_id"].apply(lambda x: x.split("/")[-2])
bad_preds_df["start_time"] = bad_preds_df["gesture_id"].apply(lambda x: float(x.split("_")[-1])/1000)
bad_preds_df

,sample_id,loss,true,pred,participant,gesture_id,session,gesture,filename,start_time
0,['/local/home/lhauptmann/data/ppg_gestures_2/p...,25.949644,6,4,berken,sp_1729175904398,2,sp,participant_berken_2.hdf5,1.729176e+09
1,['/local/home/lhauptmann/data/ppg_gestures_2/p...,20.252171,5,4,heiyi,p_1729691882407,4,p,participant_Heiyi_4.hdf5,1.729692e+09
2,['/local/home/lhauptmann/data/ppg_gestures_2/p...,18.466055,5,6,bjoern,p_1729181406108,2,p,participant_bjoern_2.hdf5,1.729181e+09
3,['/local/home/lhauptmann/data/ppg_gestures_2/p...,18.158525,0,6,lilian,o_1733480165262,1,o,participant_lilian_1.hdf5,1.733480e+09
4,['/local/home/lhauptmann/data/ppg_gestures_2/p...,15.916731,0,5,maedeh,o_1733298394254,4,o,participant_maedeh_4.hdf5,1.733298e+09
...,...,...,...,...,...,...,...,...,...,...
1616,['/local/home/lhauptmann/data/ppg_gestures_2/p...,0.721375,1,4,miro,a_1732890280614,1,a,participant_miro_1.hdf5,1.732890e+09
1617,['/local/home/lhauptmann/data/ppg_gestures_2/p...,0.719113,3,1,annamalai,c_1729268709904,4,c,participant_annamalai_4.hdf5,1.729269e+09
1618,['/local/home/lhauptmann/data/ppg_gestures_2/p...,0.717394,0,5,athena,po_1731689793673,3,po,participant_athena_3.hdf5,1.731690e+09
1619,['/local/home/lhauptmann/data/ppg_gestures_2/p...,0.712202,3,1,liuxin,c_1729610041678,3,c,participant_liuxin_3.hdf5,1.729610e+09


In [62]:
partcipant_list = pd.read_excel(r"C:\Users\lhauptmann\Code\WristPPG2\data\Recording_protocol_2.xlsx", sheet_name="recordings")
partcipant_list["file_name"] = partcipant_list.apply(lambda x: f"participant_{x['Name'].lower()}_{int(x['Session_ID'])}.hdf5", axis=1)
partcipant_list = partcipant_list[["Name", "file_name", "Session_ID", "Recording_ID"]]
partcipant_list


,Name,file_name,Session_ID,Recording_ID
0,Lars,participant_lars_1.hdf5,1,1
1,Lars,participant_lars_2.hdf5,2,2
2,Lars,participant_lars_3.hdf5,3,3
3,Lars,participant_lars_4.hdf5,4,4
4,Dominik,participant_dominik_1.hdf5,1,5
...,...,...,...,...
268,Gonzalo,participant_gonzalo_4.hdf5,4,271
269,Mollyagain,participant_mollyagain_1.hdf5,1,272
270,Mollyagain,participant_mollyagain_2.hdf5,2,273
271,Mollyagain,participant_mollyagain_3.hdf5,3,274


In [238]:

with h5py.File(data_file, "r") as f:
    data_labels = [(key.split("_")[0], float(key.split("_")[1])/1000, f[key]["start_time"][()]) for key in f.keys() if key != "data"]

In [239]:
df_lab = pd.DataFrame(data_labels, columns=["gesture", "start_time_file", "start_time"])
df_lab.sort_values(by=["start_time"], inplace=True)
df_lab.reset_index(drop=True, inplace=True)
df_lab

,gesture,start_time_file,start_time
0,s,1.733480e+09,1.733480e+09
1,p,1.733480e+09,1.733480e+09
2,d,1.733480e+09,1.733480e+09
3,p,1.733480e+09,1.733480e+09
4,o,1.733480e+09,1.733480e+09
...,...,...,...
340,o,1.733480e+09,1.733480e+09
341,o,1.733480e+09,1.733480e+09
342,o,1.733480e+09,1.733480e+09
343,s,1.733480e+09,1.733480e+09


In [259]:
bad_pred_samples = bad_preds_df[(bad_preds_df["participant"].isin(participants_with_video)) & (bad_preds_df["gesture"] != "o")]

for idx, bad_pred_sample in bad_pred_samples.iterrows():
    recording_id = partcipant_list[partcipant_list["file_name"] == bad_pred_sample["filename"]]["Recording_ID"].values[0]


    video_file = f"C:/Users/lhauptmann/Code/WristPPG2/data/webcam_recordings/webcam_{recording_id}.avi"
    label_file = f"C:/Users/lhauptmann/Code/WristPPG2/data/labels/label_{recording_id}.csv"
    data_file = f"C:/Users/lhauptmann/Code/WristPPG2/data/dataset/participant_{bad_pred_sample['participant']}/{bad_pred_sample['filename']}"
    #print(video_file, label_file, data_file)

    with h5py.File(data_file, "r") as f:
        start_time_processed = min([f[key]["start_time"][()] for key in f.keys() if "s_" in key])
        
    label_df = pd.read_csv(label_file)
    start_time_orig = label_df[label_df["label"] == "s"]["start_time"].min()
    shift = start_time_processed - start_time_orig
    bad_label_orig = label_df


    bad_label_orig = label_df
    print(bad_pred_sample["gesture"])
    #bad_label_orig = bad_label_orig[bad_label_orig["start_time"] <= bad_pred_sample["start_time"] - shift].iloc[-1]
    bad_label_orig = bad_label_orig[abs(bad_label_orig["start_time"] + shift - bad_pred_sample["start_time"]) < 0.1]
    if len(bad_label_orig) == 1:
        start_time, end_time = bad_label_orig["start_time"].values[0], bad_label_orig["end_time"].values[0]
    else:
        start_time, end_time = bad_pred_sample["start_time"] - shift, bad_pred_sample["start_time"] - shift + 1.5
        
    print(start_time, end_time)
    print(bad_pred_sample["true"] , "  -->  ", bad_pred_sample["pred"])
    try:
        bad_video = load_frames_by_time_range(video_file,start_time, end_time, offset=0, delay_frames=0)
        show_video_from_frames_pygame(bad_video, fps=30)
    except:
        print("Error")

b
1733222866.819645 1733222868.513031
2   -->   3
p
1733485662.6548915 1733485663.9992092
5   -->   2
b
1733318992.097839 1733318993.2328594
2   -->   6
c
1733236847.456478 1733236849.0663075
3   -->   4
b
1732873381.8254948 1732873382.976695
2   -->   1
b
1733476390.6555607 1733476391.863797
2   -->   6
po
1733298367.8955967 1733298369.228732
0   -->   5
c
1733318936.9819915 1733318938.4974172
3   -->   4
b
1733319048.3093498 1733319049.7687883
2   -->   3
d
1733298301.6803515 1733298302.9166045
4   -->   3
b
1733485574.860865 1733485576.1621454
2   -->   4
a
1732699476.8023002 1732699478.238081
1   -->   0
Error
c
1733479897.9015214 1733479899.317813
3   -->   6
b
1733485659.8282788 1733485661.2455838
2   -->   1
p
1733394414.6394422 1733394415.8486986
5   -->   1
p
1733394341.7015393 1733394343.0899749
5   -->   0
sp
1733222831.7931352 1733222833.3974893
6   -->   3
c
1732873521.3180492 1732873522.8098888
3   -->   0
b
1733232908.877892 1733232910.313135
2   -->   1
sp
1733394399.52

o
1733415547.163749 1733415548.663749
0   -->   1


True

In [209]:
bad_label_orig["start_time"] - bad_pred_sample["start_time"]

0     -130.479100
1     -125.472606
2     -124.009953
3     -122.775024
4     -121.486030
          ...    
152    130.711321
153    132.014215
154    133.463154
155    134.688922
156    169.795209
Name: start_time, Length: 157, dtype: float64

In [207]:
bad_pred_sample["start_time"]

1732699449.622

In [206]:
abs(bad_label_orig["start_time"]  - bad_pred_sample["start_time"])

0      130.479100
1      125.472606
2      124.009953
3      122.775024
4      121.486030
          ...    
152    130.711321
153    132.014215
154    133.463154
155    134.688922
156    169.795209
Name: start_time, Length: 157, dtype: float64

In [144]:
LETTER_GESTURES

sample_id      ['/local/home/lhauptmann/data/ppg_gestures_2/p...
loss                                                   18.158525
true                                                           0
pred                                                           6
participant                                               lilian
gesture_id                                       o_1733480165262
session                                                        1
gesture                                                        o
filename                               participant_lilian_1.hdf5
start_time                                        1733480165.262
Name: 3, dtype: object